In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score



from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

from scipy.stats import uniform, randint, loguniform
from sklearn.model_selection import RepeatedStratifiedKFold

In [2]:
sns.set_style("whitegrid", {'grid.linestyle': '--'})
sns.set_style({'font.family':'serif', 'font.serif':'Computer Modern'})
sns.set_context(font_scale=2, rc={"font.size":10,"axes.titlesize":20,"axes.labelsize":15})

In [3]:
df_train = pd.read_csv("df_train_processed.csv")
df_test = pd.read_csv("df_test_processed.csv")

In [4]:
df_train.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,...,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,1,M,158558,0.145081,9.150000e-07,...,0.155455,0.000000,0.000000,0.566462,0.709962,0.799141,0.896606,1.0,6.742190,-1.706215
1,speech,neutral,normal,Kids are talking by the door,2nd,1,M,160160,0.114319,7.137790e-07,...,0.163183,0.270133,0.368623,0.472736,0.623183,0.744908,0.874713,1.0,-0.700420,-0.201495
2,speech,neutral,normal,Dogs are sitting by the door,1st,1,M,156956,0.149963,9.554485e-07,...,0.190634,0.000000,0.000000,0.417919,0.643636,0.774253,0.899156,1.0,1.688986,-1.024773
3,speech,neutral,normal,Dogs are sitting by the door,2nd,1,M,152152,0.139618,9.176213e-07,...,0.182551,0.205616,0.306280,0.399641,0.606910,0.755213,0.886474,1.0,-0.594111,-0.412871
4,speech,calm,normal,Kids are talking by the door,1st,1,M,169769,0.137665,8.108948e-07,...,0.177069,0.177847,0.248765,0.428202,0.634815,0.759914,0.878014,1.0,0.126535,-0.620782


In [5]:
df_test.head()

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,frame_count,sum,mean,...,stft_std_w4,stft_min_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,speech,neutral,normal,Kids are talking by the door,1st,19,M,161762,0.475616,2.940224e-06,...,0.169929,0.222808,0.306697,0.466157,0.645931,0.789123,0.908316,1.0,-0.260279,-0.566556
1,speech,neutral,normal,Kids are talking by the door,2nd,19,M,155355,0.120148,7.733752e-07,...,0.216928,0.114567,0.170786,0.279347,0.557524,0.733053,0.866973,1.0,-0.428900,-0.587210
2,speech,neutral,normal,Dogs are sitting by the door,1st,19,M,160160,-0.599304,-3.741909e-06,...,0.227178,0.010295,0.023285,0.305395,0.577750,0.741576,0.889984,1.0,0.626916,-0.883317
3,speech,neutral,normal,Dogs are sitting by the door,2nd,19,M,158558,-0.080383,-5.069646e-07,...,0.179467,0.241405,0.368081,0.434591,0.620144,0.776419,0.904668,1.0,-0.825144,-0.383701
4,speech,calm,normal,Kids are talking by the door,1st,19,M,176176,0.152222,8.640319e-07,...,0.189124,0.217860,0.298851,0.393112,0.623309,0.772964,0.907035,1.0,-0.536669,-0.547354


## Standardization

## Train

In [6]:
df_train_num = df_train.select_dtypes(include="number").drop('actor', axis=1)
df_train_cat = df_train.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_train_num_scaled = pd.DataFrame(scaler.fit_transform(df_train_num), columns=df_train_num.columns)
df_train = pd.concat([df_train_num_scaled, df_train_cat], axis=1)

# Test

In [7]:
df_test_num = df_test.select_dtypes(include="number").drop('actor', axis=1)
df_test_cat = df_test.select_dtypes(include=['object'])

scaler = MinMaxScaler()
df_test_num_scaled = pd.DataFrame(scaler.fit_transform(df_test_num), columns=df_test_num.columns)
df_test = pd.concat([df_test_num_scaled, df_test_cat], axis=1)

In [8]:
def label_Encoder(array, label):
    new_array = np.zeros(len(array))
    for i in range(len(array)):
        if array[i] == label:
            new_array[i] = 1
    return new_array

## Vocal Channel

In [9]:
df_train_model_vc = df_train.copy()

In [10]:
df_test_model_vc = df_test.copy()

In [11]:
X_train_vc = df_train_model_vc.drop("vocal_channel", axis=1)
X_train_vc = np.array(pd.get_dummies(X_train_vc))
y_train_vc = label_Encoder(df_train_model_vc["vocal_channel"], 'speech')

In [12]:
X_train_vc.shape

(1828, 392)

In [13]:
X_test_vc = df_test_model_vc.drop("vocal_channel", axis=1)
X_test_vc = np.array(pd.get_dummies(X_test_vc))
y_test_vc = label_Encoder(df_test_model_vc["vocal_channel"], 'speech')

In [14]:
def bagging_classifier(X_train_array, y_train_array):
    
    
    param_grid = { "estimator__min_samples_split": loguniform(1e-2, 1e0),
                  "estimator__min_samples_leaf": uniform(0.001, 0.2),
                   "estimator__max_depth": list(range(2,200)),
                   "estimator__criterion": ["gini", "entropy", "log_loss"],
                  'max_samples':[0.5, 0.6, 0.7, 0.8], 
                  'max_features': np.arange(2,X_train_array.shape[1]),
    }


    clf=BaggingClassifier(DecisionTreeClassifier(), n_estimators = 100, random_state=42,)


    baggingclassifier=RandomizedSearchCV(clf,
                                    param_grid,
                                    cv=3,
                                    verbose=1,
                                    n_jobs=None,
                                    n_iter = 100,
                                    error_score='raise'
                      )


    baggingclassifier.fit(X_train_array,y_train_array)

    return baggingclassifier

## Bagging Classifier with Decision Tree

In [15]:
baggingclassifier_vocal_channel = bagging_classifier(X_train_vc, y_train_vc)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [16]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_vocal_channel.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'entropy', 'estimator__max_depth': 58, 'estimator__min_samples_leaf': 0.007478684666143164, 'estimator__min_samples_split': 0.022860230988955524, 'max_features': 330, 'max_samples': 0.7}


In [17]:
model_vocal_channel = baggingclassifier_vocal_channel.best_estimator_
model_vocal_channel.fit(X_train_vc,y_train_vc)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                   max_depth=58,
                                                   min_samples_leaf=0.007478684666143164,
                                                   min_samples_split=0.022860230988955524),
                  max_features=330, max_samples=0.7, n_estimators=100,
                  random_state=42)

In [18]:
y_pred_dt_vc = model_vocal_channel.predict(X_test_vc)

In [19]:
print(classification_report(y_test_vc, y_pred_dt_vc))

              precision    recall  f1-score   support

         0.0       0.90      0.99      0.94       264
         1.0       0.99      0.92      0.96       360

    accuracy                           0.95       624
   macro avg       0.95      0.96      0.95       624
weighted avg       0.95      0.95      0.95       624



In [20]:
accuracy_bag_vc = accuracy_score(y_test_vc, y_pred_dt_vc)
f1_score_bag_vc = f1_score(y_test_vc, y_pred_dt_vc, average = 'weighted')

## Sex

In [21]:
df_train_model_s = df_train.copy()

In [22]:
df_test_model_s = df_test.copy()

In [23]:
X_train_s = df_train_model_s.drop("sex", axis=1)
X_train_s = np.array(pd.get_dummies(X_train_s))
y_train_s = label_Encoder(df_train_model_s["sex"], 'M')

In [24]:
X_test_s = df_test_model_s.drop("sex", axis=1)
X_test_s = np.array(pd.get_dummies(X_test_s))
y_test_s = label_Encoder(df_test_model_s["sex"], 'M')

## Bagging Classifier with Decision Tree

In [25]:
baggingclassifier_sex = bagging_classifier(X_train_s, y_train_s)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [26]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_sex.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 51, 'estimator__min_samples_leaf': 0.0028334011398623007, 'estimator__min_samples_split': 0.01046870162115209, 'max_features': 78, 'max_samples': 0.7}


In [27]:
model_sex = baggingclassifier_sex.best_estimator_
model_sex.fit(X_train_s,y_train_s)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                   max_depth=51,
                                                   min_samples_leaf=0.0028334011398623007,
                                                   min_samples_split=0.01046870162115209),
                  max_features=78, max_samples=0.7, n_estimators=100,
                  random_state=42)

In [28]:
y_pred_dt_s = model_sex.predict(X_test_s)

In [29]:
print(classification_report(y_test_s, y_pred_dt_s))

              precision    recall  f1-score   support

         0.0       0.98      0.77      0.86       312
         1.0       0.81      0.99      0.89       312

    accuracy                           0.88       624
   macro avg       0.90      0.88      0.88       624
weighted avg       0.90      0.88      0.88       624



In [30]:
accuracy_bag_s = accuracy_score(y_test_s, y_pred_dt_s)
f1_score_bag_s = f1_score(y_test_s, y_pred_dt_s, average = 'weighted')

## Emotional Intensity

In [31]:
df_train_model_ei = df_train.copy()

In [32]:
df_test_model_ei = df_test.copy()

In [33]:
X_train_ei = df_train_model_ei.drop("emotional_intensity", axis=1)
X_train_ei = np.array(pd.get_dummies(X_train_ei))
y_train_ei = label_Encoder(df_train_model_ei["emotional_intensity"], 'normal')

In [34]:
X_test_ei = df_test_model_ei.drop("emotional_intensity", axis=1)
X_test_ei = np.array(pd.get_dummies(X_test_ei))
y_test_ei = label_Encoder(df_test_model_ei["emotional_intensity"], 'normal')

## Bagging Classifier with Decision Tree

In [35]:
baggingclassifier_emotional_intensity = bagging_classifier(X_train_ei, y_train_ei)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [36]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_emotional_intensity.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'gini', 'estimator__max_depth': 16, 'estimator__min_samples_leaf': 0.016747742726909487, 'estimator__min_samples_split': 0.013545506717983444, 'max_features': 381, 'max_samples': 0.6}


In [37]:
model_emotional_intensity = baggingclassifier_emotional_intensity.best_estimator_
model_emotional_intensity.fit(X_train_ei,y_train_ei)

BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=16,
                                                   min_samples_leaf=0.016747742726909487,
                                                   min_samples_split=0.013545506717983444),
                  max_features=381, max_samples=0.6, n_estimators=100,
                  random_state=42)

In [38]:
y_pred_dt_ei = model_emotional_intensity.predict(X_test_ei)

In [39]:
print(classification_report(y_test_ei, y_pred_dt_ei))

              precision    recall  f1-score   support

         0.0       0.75      0.69      0.72       288
         1.0       0.75      0.81      0.78       336

    accuracy                           0.75       624
   macro avg       0.75      0.75      0.75       624
weighted avg       0.75      0.75      0.75       624



In [40]:
accuracy_bag_ei = accuracy_score(y_test_ei, y_pred_dt_ei)
f1_score_bag_ei = f1_score(y_test_ei, y_pred_dt_ei, average = 'weighted')

## Emotion

In [41]:
def label_encoder_e(array):
    new_array = np.zeros(len(array))
    for i in range(len(array)):
        if array[i] == "calm":
            new_array[i] = 1
        elif array[i] == "happy":
            new_array[i] = 2
        elif array[i] == "sad":
            new_array[i] = 3
        elif array[i] == "angry":
            new_array[i] = 4
        elif array[i] == "fearful":
            new_array[i] = 5
        elif array[i] == "disgust":
            new_array[i] = 6
        elif array[i] == "surprised":
            new_array[i] = 7
    return new_array

In [42]:
df_train_model_e = df_train.copy()

In [43]:
df_test_model_e = df_test.copy()

In [44]:
X_train_e = df_train_model_e.drop("emotion", axis=1)
X_train_e = np.array(pd.get_dummies(X_train_e))
y_train_e = label_encoder_e(df_train_model_e["emotion"])

In [45]:
X_test_e = df_test_model_e.drop("emotion", axis=1)
X_test_e = np.array(pd.get_dummies(X_test_e))
y_test_e = label_encoder_e(df_test_model_ei["emotion"])

## Bagging Classifier with Decision Tree

In [46]:
baggingclassifier_emotion = bagging_classifier(X_train_e, y_train_e)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [47]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_emotion.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'entropy', 'estimator__max_depth': 23, 'estimator__min_samples_leaf': 0.0017632902987803503, 'estimator__min_samples_split': 0.027396329997694435, 'max_features': 126, 'max_samples': 0.7}


In [48]:
model_emotion = baggingclassifier_emotion.best_estimator_
model_emotion.fit(X_train_e,y_train_e)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='entropy',
                                                   max_depth=23,
                                                   min_samples_leaf=0.0017632902987803503,
                                                   min_samples_split=0.027396329997694435),
                  max_features=126, max_samples=0.7, n_estimators=100,
                  random_state=42)

In [49]:
y_pred_dt_e = model_emotion.predict(X_test_e)

In [50]:
print(classification_report(y_test_e, y_pred_dt_e))

              precision    recall  f1-score   support

         0.0       0.46      0.23      0.31        48
         1.0       0.47      0.75      0.58        96
         2.0       0.33      0.40      0.36        96
         3.0       0.29      0.08      0.13        96
         4.0       0.42      0.79      0.55        96
         5.0       0.69      0.25      0.37        96
         6.0       0.41      0.33      0.37        48
         7.0       0.41      0.42      0.41        48

    accuracy                           0.42       624
   macro avg       0.43      0.41      0.38       624
weighted avg       0.44      0.42      0.39       624



In [51]:
accuracy_bag_e = accuracy_score(y_test_e, y_pred_dt_e)
f1_score_bag_e = f1_score(y_test_e, y_pred_dt_e, average = 'weighted')

## Some Emotion

In [52]:
df_train_model_es = df_train.copy()

In [53]:
df_test_model_es = df_test.copy()

In [54]:
df_train_subset = df_train_model_es[(df_train_model_es["emotion"] == 'happy')] 
df_train_subset1 = df_train_model_es[(df_train_model_es["emotion"] == 'sad')]
df_train_subset_e = pd.concat([df_train_subset, df_train_subset1])

In [55]:
label_encoder_es = preprocessing.LabelEncoder()
X_train_es = df_train_subset_e.drop("emotion", axis=1)
X_train_es = np.array(pd.get_dummies(X_train_es))
y_train_es = label_encoder_es.fit_transform(df_train_subset_e['emotion'])

In [56]:
df_test_subset = df_test_model_es[(df_test_model_es["emotion"] == 'happy')] 
df_test_subset1 = df_test_model_es[(df_test_model_es["emotion"] == 'sad')]
df_test_subset_e = pd.concat([df_test_subset, df_test_subset1])

In [57]:
X_test_es = df_test_subset_e.drop("emotion", axis=1)
X_test_es = np.array(pd.get_dummies(X_test_es))
y_test_es = label_encoder_es.fit_transform(df_test_subset_e["emotion"])

## Bagging Classifier with Decision Tree

In [58]:
baggingclassifier_emotion_subset = bagging_classifier(X_train_es, y_train_es)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [59]:
print("Tuned Random Forest Parameters: {}".format(baggingclassifier_emotion_subset.best_params_))

Tuned Random Forest Parameters: {'estimator__criterion': 'log_loss', 'estimator__max_depth': 166, 'estimator__min_samples_leaf': 0.0013128317397892878, 'estimator__min_samples_split': 0.029615641854197756, 'max_features': 119, 'max_samples': 0.5}


In [60]:
model_emotion_subset = baggingclassifier_emotion_subset.best_estimator_
model_emotion_subset.fit(X_train_es,y_train_es)

BaggingClassifier(estimator=DecisionTreeClassifier(criterion='log_loss',
                                                   max_depth=166,
                                                   min_samples_leaf=0.0013128317397892878,
                                                   min_samples_split=0.029615641854197756),
                  max_features=119, max_samples=0.5, n_estimators=100,
                  random_state=42)

In [61]:
y_pred_dt_es = model_emotion_subset.predict(X_test_es)

In [62]:
print(classification_report(y_test_es, y_pred_dt_es))

              precision    recall  f1-score   support

           0       0.77      0.84      0.81        96
           1       0.83      0.75      0.79        96

    accuracy                           0.80       192
   macro avg       0.80      0.80      0.80       192
weighted avg       0.80      0.80      0.80       192



## ADABOOST

In [63]:
def adaboost_classifier(X_train_array, y_train_array):
    
    
    param_grid = { 
                  'learning_rate':[0.0001, 0.001, 0.01, 0.1, 1.0]
    }


    clf=AdaBoostClassifier(n_estimators = 100, random_state=42)


    adaboostclassifier=RandomizedSearchCV(clf,
                                    param_grid,
                                    cv=3,
                                    verbose=1,
                                    n_jobs=None,
                                    #n_iter = 100,
                                    error_score='raise'
                      )


    adaboostclassifier.fit(X_train_array,y_train_array)

    return adaboostclassifier

## Vocal Channel

In [64]:
adaboostclassifier_vocal_channel = adaboost_classifier(X_train_vc, y_train_vc)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/blertalleshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [65]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_vocal_channel.best_params_))

Tuned Random Forest Parameters: {'learning_rate': 0.0001}


In [66]:
model_adaboost_vc = adaboostclassifier_vocal_channel.best_estimator_
model_adaboost_vc.fit(X_train_vc,y_train_vc)

AdaBoostClassifier(estimator=DecisionTreeClassifier(), learning_rate=0.0001,
                   n_estimators=100, random_state=42)

In [67]:
y_adaboost_vc = model_adaboost_vc.predict(X_test_vc)

In [68]:
print(classification_report(y_test_vc, y_adaboost_vc))

              precision    recall  f1-score   support

         0.0       0.87      0.87      0.87       264
         1.0       0.90      0.91      0.90       360

    accuracy                           0.89       624
   macro avg       0.89      0.89      0.89       624
weighted avg       0.89      0.89      0.89       624



In [69]:
accuracy_ab_vc = accuracy_score(y_test_vc, y_adaboost_vc)
f1_score_ab_vc = f1_score(y_test_vc, y_adaboost_vc, average = 'weighted')

## Sex

In [70]:
adaboostclassifier_sex = adaboost_classifier(X_train_s, y_train_s)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/blertalleshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [71]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_sex.best_params_))

Tuned Random Forest Parameters: {'learning_rate': 0.0001}


In [72]:
model_adaboost_s = adaboostclassifier_sex.best_estimator_
model_adaboost_s.fit(X_train_s,y_train_s)

AdaBoostClassifier(estimator=DecisionTreeClassifier(), learning_rate=0.0001,
                   n_estimators=100, random_state=42)

In [73]:
y_adaboost_s = model_adaboost_s.predict(X_test_s)

In [74]:
print(classification_report(y_test_s, y_adaboost_s))

              precision    recall  f1-score   support

         0.0       0.92      0.72      0.81       312
         1.0       0.77      0.94      0.85       312

    accuracy                           0.83       624
   macro avg       0.85      0.83      0.83       624
weighted avg       0.85      0.83      0.83       624



In [75]:
accuracy_ab_s = accuracy_score(y_test_s, y_adaboost_s)
f1_score_ab_s = f1_score(y_test_s, y_adaboost_s, average = 'weighted')

## Emotional Intensity

In [76]:
adaboostclassifier_emotional_intensity = adaboost_classifier(X_train_ei, y_train_ei)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/blertalleshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [77]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_emotional_intensity.best_params_))

Tuned Random Forest Parameters: {'learning_rate': 0.0001}


In [78]:
model_adaboost_ei = adaboostclassifier_emotional_intensity.best_estimator_
model_adaboost_ei.fit(X_train_ei,y_train_ei)

AdaBoostClassifier(estimator=DecisionTreeClassifier(), learning_rate=0.0001,
                   n_estimators=100, random_state=42)

In [79]:
y_adaboost_ei = model_adaboost_ei.predict(X_test_ei)

In [80]:
print(classification_report(y_test_ei, y_adaboost_ei))

              precision    recall  f1-score   support

         0.0       0.61      0.64      0.62       288
         1.0       0.68      0.65      0.66       336

    accuracy                           0.64       624
   macro avg       0.64      0.64      0.64       624
weighted avg       0.65      0.64      0.64       624



In [81]:
accuracy_ab_ei = accuracy_score(y_test_ei, y_adaboost_ei)
f1_score_ab_ei = f1_score(y_test_ei, y_adaboost_ei, average = 'weighted')

## Emotion

In [82]:
adaboostclassifier_emotion = adaboost_classifier(X_train_e, y_train_e)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/blertalleshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [83]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_emotion.best_params_))

Tuned Random Forest Parameters: {'learning_rate': 0.0001}


In [84]:
model_adaboost_e = adaboostclassifier_emotion.best_estimator_
model_adaboost_e.fit(X_train_e,y_train_e)

AdaBoostClassifier(estimator=DecisionTreeClassifier(), learning_rate=0.0001,
                   n_estimators=100, random_state=42)

In [85]:
y_adaboost_e = model_adaboost_e.predict(X_test_e)

In [86]:
print(classification_report(y_test_e, y_adaboost_e))

              precision    recall  f1-score   support

         0.0       0.34      0.25      0.29        48
         1.0       0.43      0.29      0.35        96
         2.0       0.19      0.18      0.18        96
         3.0       0.25      0.20      0.22        96
         4.0       0.46      0.68      0.55        96
         5.0       0.22      0.17      0.19        96
         6.0       0.23      0.52      0.32        48
         7.0       0.16      0.12      0.14        48

    accuracy                           0.30       624
   macro avg       0.29      0.30      0.28       624
weighted avg       0.30      0.30      0.29       624



In [87]:
accuracy_ab_e = accuracy_score(y_test_e, y_adaboost_e)
f1_score_ab_e = f1_score(y_test_e, y_adaboost_e, average = 'weighted')

## Some Emotion

In [88]:
adaboostclassifier_emotion_subset = adaboost_classifier(X_train_es, y_train_es)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/Users/blertalleshi/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [89]:
print("Tuned Random Forest Parameters: {}".format(adaboostclassifier_emotion_subset.best_params_))

Tuned Random Forest Parameters: {'learning_rate': 0.0001}


In [142]:
model_adaboost_es = adaboostclassifier_emotion_subset.best_estimator_
model_adaboost_es.fit(X_train_es,y_train_es)

AdaBoostClassifier(estimator=DecisionTreeClassifier(), learning_rate=0.0001,
                   n_estimators=100, random_state=42)

In [143]:
y_adaboost_es = model_adaboost_es.predict(X_test_es)

In [144]:
print(classification_report(y_test_es, y_adaboost_es))

              precision    recall  f1-score   support

           0       0.53      0.56      0.55        96
           1       0.54      0.51      0.52        96

    accuracy                           0.54       192
   macro avg       0.54      0.54      0.54       192
weighted avg       0.54      0.54      0.54       192



## Metriche per Bagging Classifier

In [92]:
# vocal_channel

vocal_channel_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
vocal_channel_metrics_b.loc["Accuracy"] = [None, None, None, None]
vocal_channel_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
vocal_channel_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [93]:
vocal_channel_metrics_b.loc["Accuracy"][2] = accuracy_bag_vc
vocal_channel_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_vc

In [95]:
vocal_channel_metrics_b.to_csv('vocal_channel_metric_bag.csv', index=False)

In [96]:
y_score = model_vocal_channel.predict_proba(X_test_vc)
fpr, tpr, _ = roc_curve(y_test_vc, y_score[:, 1], pos_label=1)
roc_auc = auc(fpr, tpr)

In [97]:
vocal_channel_roc_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
vocal_channel_roc_b.loc["fpr"] = [None, None, None, None]
vocal_channel_roc_b.loc["tpr"] = [None, None, None, None]
vocal_channel_roc_b.loc["auc"] = [None, None, None, None]
vocal_channel_roc_b

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [98]:
vocal_channel_roc_b.loc["fpr"][2] = fpr
vocal_channel_roc_b.loc["tpr"][2] = tpr
vocal_channel_roc_b.loc["auc"][2] = roc_auc

In [99]:
vocal_channel_roc_b.to_csv('vocal_channel_score_bag.csv', index=False)

In [100]:
# sex

sex_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
sex_metrics_b.loc["Accuracy"] = [None, None, None, None]
sex_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
sex_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [101]:
sex_metrics_b.loc["Accuracy"][2] = accuracy_bag_s
sex_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_s

In [102]:
sex_metrics_b.to_csv('sex_metrics_bag.csv', index=False)

In [150]:
y_score_s_bag = model_sex.predict_proba(X_test_s)
fpr_s_bag, tpr_s_bag, _ = roc_curve(y_test_s, y_score_s_bag[:, 1],pos_label=1)
roc_auc_s_bag = auc(fpr_s_bag, tpr_s_bag)

In [151]:
sex_roc_bag = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
sex_roc_bag.loc["fpr"] = [None, None, None, None]
sex_roc_bag.loc["tpr"] = [None, None, None, None]
sex_roc_bag.loc["auc"] = [None, None, None, None]
sex_roc_bag

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [152]:
sex_roc_bag.loc["fpr"][2] = fpr_s_bag
sex_roc_bag.loc["tpr"][2] = tpr_s_bag
sex_roc_bag.loc["auc"][2] = roc_auc_s_bag

In [153]:
sex_roc_bag.to_csv('sex_score_bag.csv', index=False)

In [154]:
# emotional_intensity

emotional_intensity_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotional_intensity_metrics_b.loc["Accuracy"] = [None, None, None, None]
emotional_intensity_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotional_intensity_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [155]:
emotional_intensity_metrics_b.loc["Accuracy"][2] = accuracy_bag_ei
emotional_intensity_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_ei

In [156]:
emotional_intensity_metrics_b.to_csv('emotional_intensity_metrics_bag.csv', index=False)

In [157]:
y_score_ei_bag = model_emotional_intensity.predict_proba(X_test_ei)
fpr_ei_bag, tpr_ei_bag, _ = roc_curve(y_test_ei, y_score_ei_bag[:, 1],pos_label=1)
roc_auc_ei_bag = auc(fpr_ei_bag, tpr_ei_bag)

In [158]:
# emotional_intensity

emotional_intensity_roc_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
emotional_intensity_roc_b.loc["fpr"] = [None, None, None, None]
emotional_intensity_roc_b.loc["tpr"] = [None, None, None, None]
emotional_intensity_roc_b.loc["auc"] = [None, None, None, None]
emotional_intensity_roc_b

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [159]:
emotional_intensity_roc_b.loc["fpr"][2] = fpr_ei_bag
emotional_intensity_roc_b.loc["tpr"][2] = tpr_ei_bag
emotional_intensity_roc_b.loc["auc"][2] = roc_auc_ei_bag

In [160]:
emotional_intensity_roc_b.to_csv('emotional_intensity_scores_bag.csv',index=False)

In [161]:
# emotion

emotion_metrics_b = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotion_metrics_b.loc["Accuracy"] = [None, None, None, None]
emotion_metrics_b.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotion_metrics_b

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [162]:
emotion_metrics_b.loc["Accuracy"][2] = accuracy_bag_e
emotion_metrics_b.loc["F1-Score (Weighted-Average)"][2] = f1_score_bag_e

In [163]:
emotion_metrics_b.to_csv('emotion_metrics_bag.csv', index=False)

## Metriche per Adaboost

In [117]:
# vocal_channel

vocal_channel_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
vocal_channel_metrics_ab.loc["Accuracy"] = [None, None, None, None]
vocal_channel_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
vocal_channel_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [118]:
vocal_channel_metrics_ab.loc["Accuracy"][3] = accuracy_ab_vc
vocal_channel_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_vc

In [119]:
vocal_channel_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,0.889423
F1-Score (Weighted-Average),None,None,None,0.889395


In [120]:
vocal_channel_metrics_ab.to_csv('vocal_channel_metric_ab.csv', index=False)

In [164]:
y_score_ab = model_adaboost_vc.predict_proba(X_test_vc)
fpr_ab, tpr_ab, _ = roc_curve(y_test_vc, y_score_ab[:, 1],pos_label=1)
roc_auc_ab = auc(fpr_ab, tpr_ab)

In [165]:
vocal_channel_roc_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
vocal_channel_roc_ab.loc["fpr"] = [None, None, None, None]
vocal_channel_roc_ab.loc["tpr"] = [None, None, None, None]
vocal_channel_roc_ab.loc["auc"] = [None, None, None, None]
vocal_channel_roc_ab

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [166]:
vocal_channel_roc_ab.loc["fpr"][3] = fpr_ab
vocal_channel_roc_ab.loc["tpr"][3] = tpr_ab
vocal_channel_roc_ab.loc["auc"][3] = roc_auc_ab

In [167]:
vocal_channel_roc_ab.to_csv('vocal_channel_score_ab.csv', index=False)

In [168]:
# sex

sex_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
sex_metrics_ab.loc["Accuracy"] = [None, None, None, None]
sex_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
sex_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [169]:
sex_metrics_ab.loc["Accuracy"][3] = accuracy_ab_s
sex_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_s

In [170]:
sex_metrics_ab.to_csv('sex_metric_ab.csv', index=False)

In [171]:
y_score_s_ab = model_adaboost_s.predict_proba(X_test_s)
fpr_s_ab, tpr_s_ab, _ = roc_curve(y_test_s, y_score_s_ab[:, 1],pos_label=1)
roc_auc_s_ab = auc(fpr_s_ab, tpr_s_ab)

In [172]:
sex_roc_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
sex_roc_ab.loc["fpr"] = [None, None, None, None]
sex_roc_ab.loc["tpr"] = [None, None, None, None]
sex_roc_ab.loc["auc"] = [None, None, None, None]
sex_roc_ab

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [173]:
sex_roc_ab.loc["fpr"][3] = fpr_s_ab
sex_roc_ab.loc["tpr"][3] = tpr_s_ab
sex_roc_ab.loc["auc"][3] = roc_auc_s_ab

In [174]:
sex_roc_ab.to_csv('sex_score_ab.csv', index=False)

In [175]:
# emotional_intensity

emotional_intensity_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotional_intensity_metrics_ab.loc["Accuracy"] = [None, None, None, None]
emotional_intensity_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotional_intensity_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [176]:
emotional_intensity_metrics_ab.loc["Accuracy"][3] = accuracy_ab_ei
emotional_intensity_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_ei

In [177]:
emotional_intensity_metrics_ab.to_csv('emotional_intensity_metrics_ab.csv', index=False)

In [178]:
y_score_ei_ab = model_adaboost_ei.predict_proba(X_test_ei)
fpr_ei_ab, tpr_ei_ab, _ = roc_curve(y_test_ei, y_score_ei_ab[:, 1],pos_label=1)
roc_auc_ei_ab = auc(fpr_ei_ab, tpr_ei_ab)

In [179]:
# emotional_intensity

emotional_intensity_roc_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["fpr", "tpr", "auc"])
emotional_intensity_roc_ab.loc["fpr"] = [None, None, None, None]
emotional_intensity_roc_ab.loc["tpr"] = [None, None, None, None]
emotional_intensity_roc_ab.loc["auc"] = [None, None, None, None]
emotional_intensity_roc_ab

,LG,RF,DTB,AB
fpr,None,None,None,None
tpr,None,None,None,None
auc,None,None,None,None


In [180]:
emotional_intensity_roc_ab.loc["fpr"][3] = fpr_ei_ab
emotional_intensity_roc_ab.loc["tpr"][3] = tpr_ei_ab
emotional_intensity_roc_ab.loc["auc"][3] = roc_auc_ei_ab

In [181]:
emotional_intensity_roc_ab.to_csv('emotional_intensity_scores_ab.csv', index=False)

In [182]:
# emotion

emotion_metrics_ab = pd.DataFrame(columns=["LG", "RF", "DTB", "AB"], 
                       index=["Accuracy", "F1-Score (Weighted-Average)"])
emotion_metrics_ab.loc["Accuracy"] = [None, None, None, None]
emotion_metrics_ab.loc["F1-Score (Weighted-Average)"] = [None, None, None, None]
emotion_metrics_ab

,LG,RF,DTB,AB
Accuracy,None,None,None,None
F1-Score (Weighted-Average),None,None,None,None


In [183]:
emotion_metrics_ab.loc["Accuracy"][3] = accuracy_ab_e
emotion_metrics_ab.loc["F1-Score (Weighted-Average)"][3] = f1_score_ab_e

In [184]:
emotion_metrics_ab.to_csv('emotion_metrics_ab.csv',index=False)